In [19]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, LongType

import os

In [3]:
spark = (
    SparkSession
    .builder
    .appName('Schema handling')
    .enableHiveSupport()
    .getOrCreate()
)

There are two json files with incompatible Schema, the score column is Long in the first file and Double in the second:

#### Content of file-1.json:
{"user_id": 100, "score": 50} <br>
{"user_id": 200, "score": 51}

#### Content of file-2.json:
{"user_id": 300, "score": 45.0} <br>
{"user_id": 400, "score": 46.0}

In [4]:
spark.version

'2.4.5'

In [11]:
base_path = os.getcwd()

project_path = ('/').join(base_path.split('/')[0:-1]) 

data_input_path = os.path.join(project_path, 'data/incompatible-json-schema')

In [21]:
schema = StructType([
    StructField('score', LongType()),
    StructField('user_id', LongType())
])

In [22]:
# Read with default mode value (mode='PERMISSIVE')

df = spark.read.schema(schema).json(data_input_path)

In [23]:
df.printSchema()

root
 |-- score: long (nullable = true)
 |-- user_id: long (nullable = true)



In [24]:
df.show()

+-----+-------+
|score|user_id|
+-----+-------+
| null|   null|
| null|   null|
|   50|    100|
|   51|    200|
+-----+-------+



In [25]:
df.count()

4

In [26]:
df.filter(col('score').isNotNull()).count()

2

In [27]:
df.filter(col('user_id').isNotNull()).count()

4

In [28]:
df.collect()

[Row(score=None, user_id=None),
 Row(score=None, user_id=None),
 Row(score=50, user_id=100),
 Row(score=51, user_id=200)]

In [29]:
df.select('user_id').collect()

[Row(user_id=300), Row(user_id=400), Row(user_id=100), Row(user_id=200)]

In [30]:
df.select('score').collect()

[Row(score=None), Row(score=None), Row(score=50), Row(score=51)]

In [31]:
# Read with mode value DROPMALFORMED

df = (
    spark.read
    .schema(schema)
    .option('mode', 'DROPMALFORMED')
    .json(data_input_path)
)

In [32]:
df.printSchema()

root
 |-- score: long (nullable = true)
 |-- user_id: long (nullable = true)



In [33]:
df.show()

+-----+-------+
|score|user_id|
+-----+-------+
|   50|    100|
|   51|    200|
+-----+-------+



In [34]:
df.count()

4

In [37]:
df.filter(col('score').isNotNull()).count()

2

In [38]:
df.filter(col('user_id').isNotNull()).count()

4

In [35]:
df.collect()

[Row(score=50, user_id=100), Row(score=51, user_id=200)]

In [36]:
df.select('user_id').collect()

[Row(user_id=300), Row(user_id=400), Row(user_id=100), Row(user_id=200)]

In [39]:
df.select('score').collect()

[Row(score=50), Row(score=51)]

In [40]:
df.select('user_id').show()

+-------+
|user_id|
+-------+
|    300|
|    400|
|    100|
|    200|
+-------+



The DROPMALFORMED mode leads to confusing situations, since the total count returns 4 however the collected list has only two records. Similarly the show funtions displays only 2 records, however when we select only user_id it displays for records.

In [41]:
# Read with mode value DROPMALFORMED

df = (
    spark.read
    .schema(schema)
    .option('mode', 'FAILFAST')
    .json(data_input_path)
)

In [42]:
df.count()

4

In [43]:
df.printSchema()

root
 |-- score: long (nullable = true)
 |-- user_id: long (nullable = true)



In [44]:
df.show()

Py4JJavaError: An error occurred while calling o161.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 30.0 failed 1 times, most recent failure: Lost task 0.0 in stage 30.0 (TID 45, localhost, executor driver): org.apache.spark.SparkException: Malformed records are detected in record parsing. Parse Mode: FAILFAST.
	at org.apache.spark.sql.execution.datasources.FailureSafeParser.parse(FailureSafeParser.scala:70)
	at org.apache.spark.sql.execution.datasources.json.TextInputJsonDataSource$$anonfun$readFile$2.apply(JsonDataSource.scala:143)
	at org.apache.spark.sql.execution.datasources.json.TextInputJsonDataSource$$anonfun$readFile$2.apply(JsonDataSource.scala:143)
	at scala.collection.Iterator$$anon$12.nextCur(Iterator.scala:435)
	at scala.collection.Iterator$$anon$12.hasNext(Iterator.scala:441)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileSca

In [45]:
df.select('user_id').show()

+-------+
|user_id|
+-------+
|    300|
|    400|
|    100|
|    200|
+-------+



In [47]:
df.collect()

Py4JJavaError: An error occurred while calling o161.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 34.0 failed 1 times, most recent failure: Lost task 0.0 in stage 34.0 (TID 49, localhost, executor driver): org.apache.spark.SparkException: Malformed records are detected in record parsing. Parse Mode: FAILFAST.
	at org.apache.spark.sql.execution.datasources.FailureSafeParser.parse(FailureSafeParser.scala:70)
	at org.apache.spark.sql.execution.datasources.json.TextInputJsonDataSource$$anonfun$readFile$2.apply(JsonDataSource.scala:143)
	at org.apache.spark.sql.execution.datasources.json.TextInputJsonDataSource$$anonfun$readFile$2.apply(JsonDataSource.scala:143)
	at scala.collection.Iterator$$anon$12.nextCur(Iterator.scala:435)
	at scala.collection.Iterator$$anon$12.hasNext(Iterator.scala:441)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(Fi